In [ ]:
from src.custom_agents.planner_agent import PlannerAgent

planner_agent = PlannerAgent()


In [ ]:
await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
from src.custom_agents.base import BaseAgent
import os 
agent = BaseAgent("test bot", "", os.getenv(""))
await agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI, AsyncAzureOpenAI

_ = load_dotenv()

client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
from agents import Agent, Runner, OpenAIChatCompletionsModel

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",
    model=OpenAIChatCompletionsModel(
        model=os.getenv("GPT4O_MINI_DEPLOYMENT"),
        openai_client=client,
    )
)

In [ ]:
result = await Runner.run(
    agent,
    "Make a SWOT analysis for apple Inc.",
)

In [ ]:
result.raw_responses

In [ ]:
from custom_agents.swot_agent import SWOTAgent

planner_agent = SWOTAgent()
resutl = await planner_agent.execute("Make a SWOT analysis for apple Inc.")

In [ ]:
for output in resutl.final_output:
    print(output)

In [ ]:
from custom_agents.base_agent import BaseAgent

base_agent = BaseAgent(
    name="Chatbot",
)

In [ ]:
result = await base_agent.execute("Wie alt wurde Albert Einstein am 14. März 1879?")

In [ ]:
result.final_output

In [ ]:
# from dotenv import load_dotenv
# import os
from manager import SWOTAnalysisManager

# load_dotenv()

# print(os.getenv("O3_MINI_DEPLOYMENT"))

swot_manager = SWOTAnalysisManager()

In [ ]:
result = await swot_manager.run("Make a very detailed SWOT analysis for SAP and use data from 2025.")

# Baue den SWOTAnalysisManager

## Aufbau
1. Planner
2. Searcher
3. SWOT

### 1. Planner

In [ ]:
from agents import Runner

query = "Make a very detailed SWOT analysis for SAP and use data from 2025."

In [ ]:
from custom_agents.planner_agent import PlannerAgent, WebSearchPlan, WebSearchItem
from openai.types.responses import ResponseTextDeltaEvent

planner_agent = PlannerAgent()
planner_result = planner_agent.execute_streamed(f"Query: {query}")

In [ ]:
for plan in planner_result.final_output_as(WebSearchPlan).searches:
    print(f"{plan.query} ----> {plan.reason}")

### 2. Search

In [ ]:
from custom_agents.search_agent import SearchAgent

search_agent = SearchAgent()

In [ ]:
async def search(item: WebSearchItem):
    input_data = f"Search term: {item.query}\nReason: {item.reason}\n"
    result = await search_agent.execute(input_data)
    return result.final_output
    

In [ ]:
import asyncio
from agents import custom_span

tasks = [asyncio.create_task(search(item)) for item in planner_result.final_output_as(WebSearchPlan).searches]

with custom_span("Search for all items"):
    # await asyncio.gather(*tasks)
    search_resutls = []
    for task in asyncio.as_completed(tasks):
        result = await task
        search_resutls.append(result)

In [ ]:
search_resutls

### 3. SWOT Analysis

In [ ]:
from custom_agents.swot_agent import SWOTAgent

swot_agent = SWOTAgent()

input_data = f"Original query: {query}\nSummarized search results: {search_resutls}"

swot_result = await swot_agent.execute(input_data, max_turns=20)

In [ ]:
swot_result.final_output.swot_summary

### 4. Test New Manager Class

In [ ]:
from manager import SWOTAnalysisManager

query = "Make a very detailed SWOT analysis for SAP and use data from 2025."

swot_manager = SWOTAnalysisManager()
swot_result = await swot_manager.execute_streamed(query)

In [ ]:
for event in swot_result.stream_events():
    if isinstance(event, ResponseTextDeltaEvent):
        print(event.delta.content, end="")
    else:
        print(event)

# New Idea
## Aufbau
1. Planner ohne Stream
2. Search ohne Stream
3. SWOT mit Stream

In [ ]:
from manager import SWOTAnalysisManager

agent = SWOTAnalysisManager()

query = "Make a SWOT analysis for Deutsche Telekom"

In [ ]:
planner_result = await agent.planner_agent.execute(f"Query: {query}")

In [ ]:
n = 1
planner_answer = ""
for item in planner_result.final_output.searches:
    planner_answer += f"{n}. Search Query: {item.query} <--> Reason {item.reason}\n"
    n += 1

In [ ]:
planner_answer 

# 10k-Filings Retrieval

In [ ]:
# 1. Import the necessary functions from edgartools
from edgar import *

# 2. Tell the SEC who you are
set_identity("mike@indigo.com")

# 3. Start using the library
filings = get_filings()

In [ ]:
# Competitor of SAP
competitors = {
    "Competitors_by_Sector": {
        "Apple Inc.": "AAPL",
        "Microsoft Corporation": "MSFT",
        "NVIDIA Corporation": "NVDA",
        "Broadcom Inc.": "AVGO",
        "Oracle Corporation": "ORCL",
        "Salesforce, Inc.": "CRM",
        "Cisco Systems, Inc.": "CSCO",
        "International Business Machines Corporation": "IBM",
        "Palantir Technologies Inc.": "PLTR",
        "Accenture plc": "ACN"
    },
    "Competitors_by_Industry": {
        "Salesforce, Inc.": "CRM",
        "Intuit Inc.": "INTU",
        "ServiceNow, Inc.": "NOW",
        "Adobe Inc.": "ADBE",
        "Uber Technologies, Inc.": "UBER",
        "Automatic Data Processing, Inc.": "ADP",
        "AppLovin Corporation": "APP",
        "Cadence Design Systems, Inc.": "CDNS",
        "MicroStrategy Incorporated": "MSTR",
        "Workday, Inc.": "WDAY"
    }
}

### Retrieve latest 3 10k forms

In [ ]:
# 1. Import the necessary functions from edgartools
from edgar import set_identity, get_filings, Company

def retrieve_tenk_forms(company_ticker: str, top_k: int, path: str):
    set_identity("mike@indigo.com")
    # filings = get_filings()
    company = Company(company_ticker)
    tenk_filings = company.get_filings(form="10-K")

    i = 1
    for filing in tenk_filings:
        if i <= top_k:
            filing_name = filing.attachments[1].document.split(sep=".")[0]
            filing.attachments[1].download(path=f"{path}/{filing_name}.html")
        i += 1
    

In [ ]:
top_k = 3
for company_name, ticker in competitors["Competitors_by_Sector"].items():
    print(f"Safe top {top_k} 10k forms for {company_name}")
    retrieve_tenk_forms(ticker, top_k, "./10k-data")


In [ ]:
top_k = 3
for company_name, ticker in competitors["Competitors_by_Industry"].items():
    print(f"Safe top {top_k} 10k forms for {company_name}")
    retrieve_tenk_forms(ticker, top_k, "./10k-data")

In [ ]:
retrieve_tenk_forms('CRM', 4, "./10k-data")

In [ ]:
company = Company("CRM") # Sales

In [ ]:
tenk_filings = company.get_filings(form="10-K")

In [ ]:
i = 1
for filing in tenk_filings:
    if i <= 3:
        filing_name = filing.attachments[1].document.split(sep=".")[0]
        filing.attachments[1].download(path=f"./10k-data/{filing_name}.html")
    i += 1

In [ ]:
filings.attachments[1].download("./")

In [ ]:
filing = filings.attachments[1]

In [ ]:
type(filing)

In [ ]:
type(filings)

# 10k Filings Chunking, Embedding and Storing

In [ ]:
# process_aapl_file.py

import os
from bs4 import BeautifulSoup
from ingest.chunking import chunk_text
from ingest.embedding import generate_embedding
from vectorstore.azure_search_client import upload_documents_to_index

def process_html_file(file_path: str):
    """
    Reads an HTML file, extracts text using BeautifulSoup,
    splits it into smaller chunks, computes embeddings for each chunk,
    and returns a list of documents ready to be uploaded.
    """
    # Read the HTML file from disk
    with open(file_path, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    # Use BeautifulSoup to extract plain text
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator="\n")
    
    # Split the extracted text into chunks (default: chunk_size=1000, overlap=100)
    chunks = chunk_text(text)
    documents = []
    
    # Process each chunk: compute embedding and create a document structure
    for index, chunk in enumerate(chunks):
        # Compute the embedding for this text chunk
        embedding = generate_embedding(chunk)
        # Create a unique identifier for the chunk
        doc_id = f"aapl-20240928_{index}"
        document = {
            "id": doc_id,
            "data": chunk,
            "dataVector": embedding,
            # Additional metadata can be added below as needed:
            "metadata": "10k filing for Apple Inc.",
            "year": 2024,
            "companyname": "Apple Inc."
        }
        documents.append(document)
    
    return documents

In [ ]:
# Path to your local HTML file
file_path = "aapl-20240928.html"

# Process the HTML file into chunked documents with embeddings
documents = process_html_file(file_path)
print(f"Processed {len(documents)} chunks from {file_path}.")

# Upload the processed documents to your Azure Cognitive Search index
upload_documents_to_index(documents)
print("Documents uploaded to Azure Search index.")